<a href="https://colab.research.google.com/github/heejin0223/NLPHW/blob/main/friends_bertbase_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Friends 

> data 출처 : http://doraemon.iis.sinica.edu.tw/emotionlines/download.html 

> 참고 소스 :  
https://colab.research.google.com/drive/1EMzEfTYjYLgEHjCCP1vEr9oOZLXMocGh?usp=sharing
https://colab.research.google.com/drive/1S5lWi0IeX1Y2u3swREV2DWpxNDqmmFnt?usp=sharing
https://www.coursera.org/projects/sentiment-analysis-bert

##### 최종검사 완료

1.   BertTokenizer.from_pretrained('bert-base-cased')
2.   BertForSequenceClassification.from_pretrained('bert-base-cased')
3.   Optimizer : AdamW
4.   learning_rate = 1e-5, n_epoch = 5, batch_size= 16
5.   sentence에 화자 정보 추가

##### **Settings**

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [4]:
DATA_PATH = 'gdrive/My Drive/Colab Notebooks/NLP/'
import sys
sys.path.append(DATA_PATH)

In [5]:
!pip install transformers --quiet

     |████████████████████████████████| 1.5MB 13.0MB/s 
     |████████████████████████████████| 2.9MB 67.5MB/s 
     |████████████████████████████████| 890kB 63.2MB/s 


In [6]:
import torch

from transformers import BertTokenizer, BertModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm.notebook import tqdm


##### **Dataset**

In [7]:
## friends_train

import json

data = {'speaker': [], 'utterance': [], 'emotion': []}

for dialog in json.loads(open(DATA_PATH + 'friends_train.json').read()):
  for line in dialog:
    data['speaker'].append(line['speaker'])
    data['utterance'].append(line['utterance'])
    data['emotion'].append(line['emotion'])

train_data=pd.DataFrame([data])

speaker=pd.DataFrame(train_data.speaker[0],columns=['speaker'])
utterance=pd.DataFrame(train_data.utterance[0],columns=['utterance'])
emotion=pd.DataFrame(train_data.emotion[0],columns=['emotion'])

train = pd.concat([speaker,utterance,emotion],axis=1)
print(train.shape)
train[:5]


# speaker, utterane 를 합쳐서 하나로 speutt
train['speutt'] = train['speaker'].str.capitalize() +' said that ' + '"' + train['utterance'] + '"'
train[:2]


# 청자 정보 추가 
#train['listener'] = train['speaker'].shift(-1)

# speaker, utterane, listner 를 합쳐서 하나로 speutt
#train['speutt'] = train['speaker'].str.capitalize() +' told ' + train['listener'] + ', "' + train['utterance'] + '"'
#train[:2]

(10561, 3)


,speaker,utterance,emotion,speutt
0,Chandler,also I was the point person on my companys tr...,neutral,"Chandler said that ""also I was the point perso..."
1,The Interviewer,You mustve had your hands full.,neutral,"The interviewer said that ""You mustve had you..."


In [8]:
train.groupby('emotion').size().reset_index(name = 'counts')

,emotion,counts
0,anger,513
1,disgust,240
2,fear,185
3,joy,1283
4,neutral,4752
5,non-neutral,2017
6,sadness,351
7,surprise,1220


In [9]:
## friends_dev

import json

data = {'speaker': [], 'utterance': [], 'emotion': []}

for dialog in json.loads(open(DATA_PATH + 'friends_dev.json').read()):
  for line in dialog:
    data['speaker'].append(line['speaker'])
    data['utterance'].append(line['utterance'])
    data['emotion'].append(line['emotion'])

dev_data=pd.DataFrame([data])

speaker=pd.DataFrame(dev_data.speaker[0],columns=['speaker'])
utterance=pd.DataFrame(dev_data.utterance[0],columns=['utterance'])
emotion=pd.DataFrame(dev_data.emotion[0],columns=['emotion'])

dev = pd.concat([speaker,utterance,emotion],axis=1)
print(dev.shape)


# speaker, utterane 를 합쳐서 하나로 speutt
dev['speutt'] = dev['speaker'].str.capitalize() +' said that ' + '"' + dev['utterance'] + '"'
dev[:2]

# 청자 정보 추가 
#dev['listener'] = dev['speaker'].shift(-1)

# speaker, utterane, listner 를 합쳐서 하나로 speutt
#dev['speutt'] = dev['speaker'].str.capitalize() +' told ' + dev['listener'] + ', "' + dev['utterance'] + '"'
#dev[:2]

(1178, 3)


,speaker,utterance,emotion,speutt
0,Phoebe,"Oh my God, hes lost it. Hes totally lost it.",non-neutral,"Phoebe said that ""Oh my God, hes lost it. He..."
1,Monica,What?,surprise,"Monica said that ""What?"""


In [10]:
dev.groupby('emotion').size().reset_index(name = 'counts')

,emotion,counts
0,anger,85
1,disgust,23
2,fear,29
3,joy,123
4,neutral,491
5,non-neutral,214
6,sadness,62
7,surprise,151


In [11]:
## friends_test

import json

data = {'speaker': [], 'utterance': [], 'emotion': []}

for dialog in json.loads(open(DATA_PATH + 'friends_test.json').read()):
  for line in dialog:
    data['speaker'].append(line['speaker'])
    data['utterance'].append(line['utterance'])
    data['emotion'].append(line['emotion'])

test_data=pd.DataFrame([data])

speaker=pd.DataFrame(test_data.speaker[0],columns=['speaker'])
utterance=pd.DataFrame(test_data.utterance[0],columns=['utterance'])
emotion=pd.DataFrame(test_data.emotion[0],columns=['emotion'])

test = pd.concat([speaker,utterance,emotion],axis=1)
print(test.shape)

# speaker, utterane 를 합쳐서 하나로 speutt
test['speutt'] = test['speaker'].str.capitalize() +' said that ' + '"' + test['utterance'] + '"'
test[:2]


# 청자 정보 추가 
#test['listener'] = test['speaker'].shift(-1)

# speaker, utterane, listner 를 합쳐서 하나로 speutt
#test['speutt'] = test['speaker'].str.capitalize() +' told ' + test['listener'] + ', "' + test['utterance'] + '"'
#test[:2]

(2764, 3)


,speaker,utterance,emotion,speutt
0,Mark,Why do all youre coffee mugs have numbers on ...,surprise,"Mark said that ""Why do all youre coffee mugs ..."
1,Rachel,Oh. Thats so Monica can keep track. That way ...,non-neutral,"Rachel said that ""Oh. Thats so Monica can kee..."


In [12]:
# 8개 감정 한글과 번호로 매칭
e2i_dict = dict((emo, i) for i, emo in enumerate(set(train['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}

print(e2i_dict)
print(i2e_dict)

{'sadness': 0, 'fear': 1, 'anger': 2, 'neutral': 3, 'disgust': 4, 'surprise': 5, 'non-neutral': 6, 'joy': 7}
{0: 'sadness', 1: 'fear', 2: 'anger', 3: 'neutral', 4: 'disgust', 5: 'surprise', 6: 'non-neutral', 7: 'joy'}


In [13]:
train['label'] = train.emotion.replace(e2i_dict)
dev['label'] = dev.emotion.replace(e2i_dict)
test['label'] = test.emotion.replace(e2i_dict)

train[:5]

,speaker,utterance,emotion,speutt,label
0,Chandler,also I was the point person on my companys tr...,neutral,"Chandler said that ""also I was the point perso...",3
1,The Interviewer,You mustve had your hands full.,neutral,"The interviewer said that ""You mustve had you...",3
2,Chandler,That I did. That I did.,neutral,"Chandler said that ""That I did. That I did.""",3
3,The Interviewer,So lets talk a little bit about your duties.,neutral,"The interviewer said that ""So lets talk a lit...",3
4,Chandler,My duties? All right.,surprise,"Chandler said that ""My duties? All right.""",5


In [14]:
# training과 dev 일단 합쳤다가 다시 split하자  :: imbalance때문에
total = pd.concat([train,dev])

from sklearn.model_selection import train_test_split

d_train, d_val = train_test_split(total,
                                  test_size=0.1, 
                                  random_state=42, 
                                  stratify=total.label.values)

#total['data_type'] = ['not_set']*total.shape[0]

#total.loc[X_train, 'data_type'] = 'train'
#total.loc[X_val, 'data_type'] = 'dev'

#total.groupby(['emotion', 'label', 'data_type']).count()

In [15]:
# label별 비중
label_counts = d_train.groupby('label').size().reset_index(name = 'count')
label_counts['weights'] = label_counts['count']/len(train)
class_weights = torch.FloatTensor(label_counts.weights).cuda()
class_weights

tensor([0.0352, 0.0182, 0.0509, 0.4468, 0.0224, 0.1168, 0.1901, 0.1198],
       device='cuda:0')

##### **Model**

In [16]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [17]:
pretrained_weights = 'bert-large-cased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE=1e-5, 
EPSILON=1e-8

In [18]:
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)


### train data set
# input_ids
sentences_train = d_train['speutt']
tokenized_texts_train = [tokenizer.tokenize(sent) for sent in sentences_train]
input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# attention_masks
attention_masks_train = []
for seq in input_ids_train:
    seq_mask_train = [float(i>0) for i in seq]
    attention_masks_train.append(seq_mask_train)

# labels
train_labels = d_train['label'].values

# pytorch tensor
train_inputs = torch.tensor(input_ids_train)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(attention_masks_train)


### validation data set
# input_ids
sentences_validation = d_val['speutt']
tokenized_texts_validation = [tokenizer.tokenize(sent) for sent in sentences_validation]
input_ids_validation = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_validation]
input_ids_validation = pad_sequences(input_ids_validation, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# attention_masks
attention_masks_validation = []
for seq in input_ids_validation:
    seq_mask_validation = [float(i>0) for i in seq]
    attention_masks_validation.append(seq_mask_validation)

# labels
validation_labels = d_val['label'].values

# pytorch tensor
validation_inputs = torch.tensor(input_ids_validation)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(attention_masks_validation)


In [19]:
# 배치 사이즈
batch_size = BATCH_SIZE

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [20]:
model = BertForSequenceClassification.from_pretrained(pretrained_weights,
                                                      num_labels=len(e2i_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.cuda()

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [21]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = EPOCHS

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [22]:

# 정확도 계산 함수
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [23]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
seed_val = 23
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [25]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dataloader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(validation_dataloader)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')



Epoch 1
Training loss: 1.3163020101329381
Validation loss: 1.1240500960801099
F1 Score (Weighted): 0.5806074723618552



Epoch 2
Training loss: 1.046834256032231
Validation loss: 1.0884062945842743
F1 Score (Weighted): 0.5972939379361374



Epoch 3
Training loss: 0.8660534760955603
Validation loss: 1.137252180157481
F1 Score (Weighted): 0.6004279806473086



Epoch 4
Training loss: 0.7029808924819446
Validation loss: 1.1929800232519974
F1 Score (Weighted): 0.6013929756276576



Epoch 5
Training loss: 0.5842607596863777
Validation loss: 1.2396840304136276
F1 Score (Weighted): 0.6005209839752835



#### Test data

In [26]:
### test data set
# input_ids
sentences_test = test['speutt']
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences_test]
input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# attention_masks
attention_masks_test = []
for seq in input_ids_test:
    seq_mask_test = [float(i>0) for i in seq]
    attention_masks_test.append(seq_mask_test)

# labels
test_labels = test['label'].values

# pytorch tensor
test_inputs = torch.tensor(input_ids_test)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(attention_masks_test)

In [27]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [28]:
val_loss, predictions, true_vals = evaluate(test_dataloader)
val_f1 = f1_score_func(predictions, true_vals)
val_f1

0.6041978355090072

#### cptt data

In [29]:
final_test = pd.read_csv(DATA_PATH + "/en_data.csv", sep=',')
final_test[:5]
final_test['speutt'] = final_test['speaker'].str.capitalize() +' said that "' + final_test['utterance'] + '"'
final_test[:5]

,id,i_dialog,i_utterance,speaker,utterance,speutt
0,0,0,0,Phoebe,"Alright, whadyou do with him?","Phoebe said that ""Alright, whadyou do with him?"""
1,1,0,1,Monica,Oh! You're awake!,"Monica said that ""Oh! You're awake!"""
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...,"Joey said that ""Then you gotta come clean with..."
3,3,0,3,Mr. Tribbiani,"Yeah, but this is","Mr. tribbiani said that ""Yeah, but this is"""
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!,"Joey said that ""I don't wanna hear it! Now go ..."


In [30]:
# 문장 추출
final_sentences = final_test['speutt']
final_sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in final_sentences]

# tokenizing
final_tokenized_texts = [tokenizer.tokenize(sent) for sent in final_sentences]
input_ids_final = [tokenizer.convert_tokens_to_ids(x) for x in final_tokenized_texts]
input_ids_final = pad_sequences(input_ids_final, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# initialize attention masks
attention_masks_final = []
for seq in input_ids_final:
    seq_mask_final = [float(i>0) for i in seq]
    attention_masks_final.append(seq_mask_final)     

# pytorch tensor
final_test_id = torch.tensor(final_test['id'])
final_test_inputs = torch.tensor(input_ids_final)
final_test_masks = torch.tensor(attention_masks_final)

final_test_data = TensorDataset(final_test_id, final_test_inputs, final_test_masks)
final_test_dataloader = DataLoader(final_test_data, batch_size=1)


In [31]:
#시작 시간 설정
t0 = time.time()
batch = 1

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
final_result = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(final_test_dataloader):
    # 경과 정보 표시
    if step % 1000 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(final_test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_id, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    b_id = b_id.cpu().numpy()

    result = np.concatenate((b_id, pred_flat), axis=None)
        
    final_result.append(result)    

rdf = pd.DataFrame(final_result, columns =['Id', 'Predicted'])
rdf[:5]

  Batch 1,000  of  1,623.    Elapsed: 0:00:23.


,Id,Predicted
0,0,3
1,1,6
2,2,6
3,3,3
4,4,6


In [32]:
rdf['Expected'] = rdf.Predicted.replace(i2e_dict)
rdf[:5]

,Id,Predicted,Expected
0,0,3,neutral
1,1,6,non-neutral
2,2,6,non-neutral
3,3,3,neutral
4,4,6,non-neutral


In [33]:
rdf = rdf.drop(['Predicted'], axis=1)
rdf[:5]

,Id,Expected
0,0,neutral
1,1,non-neutral
2,2,non-neutral
3,3,neutral
4,4,non-neutral


In [34]:
rdf.to_csv(DATA_PATH + 'sample.csv', index=False)